In [5]:
# import all packages 
import numpy as np
import pandas as pd
import wooldridge as woo
import statsmodels.formula.api as smf 
import patsy as pt
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.base.model as smclass
import linearmodels.iv as iv
import linearmodels as plm

In [2]:
# ch16 q2
# corn = price + income is the demand function because a country's income may affect
# the amount of corn its able to purchase, similar to the way an individual's salary would 
# impact their weekly grocery budget. 
# corn = price + rainfall + rainfall^2 is the supply function because rainfall obviously 
# alters the amount of corn a country's crops will successfully produce. 
# the econometric identification of these two equations is sound because  both the order
# and rank conditions are met given that income is present in demand but not supply while
# rainfall is included with supply and not demand. 

In [10]:
# ch16 c1
s = woo.dataWoo('SMOKE')

# (i) b1 is the coefficient of the variable cigs and would
# describe how being a cigarette smoker affects percentage
# change in income.

# (ii) i would expect that gamma5 has a negative sign, to follow the
# law of demand. Gamma6 probably has a negative sign as well,
# because local restrictions would reduce someone's ability
# to smoke cigarettes.

# (iii) te need at least one exogeneous variable that meets
# the rank and order conditions in order for this equation
# to be identified.

reg = smf.ols(formula='lincome ~ cigs + educ + age + agesq', data=s)
results = reg.fit()
print(f'results.summary(): \n{results.summary()}\n')

# (iv) b1 is estimated to be 0.0017, but has a very large p-value, so
# it is statistically insignificant and we can assume that being a
# cigarette smoker does not affect income.

reg1 = smf.ols(formula='cigs ~ educ + age + agesq + lcigpric + restaurn', data=s)
results1 = reg1.fit()
print(f'results1.summary(): \n{results1.summary()}\n')

# (v) neither lcigpric nor restaurn are significant in the reduced form.

reg_iv = iv.IV2SLS.from_formula(formula='lincome ~ 1 + [cigs ~ lcigpric + restaurn] + \
                                         age + educ + agesq', data=s)
results_iv = reg_iv.fit(cov_type='unadjusted', debiased=True)
table_iv = pd.DataFrame({'b': round(results_iv.params, 4),
                         'se': round(results_iv.std_errors, 4),
                         't': round(results_iv.tstats, 4),
                         'pval': round(results_iv.pvalues, 4)})
print(f'table_iv: \n{table_iv}\n')

# (vi) the 2SLS model estimates B1 as negative, which is more expected
# than the small positive sign in the OLS model. however, 2SLS
# still includes cigs as a barely statistically insignificant variable.

# (vii) i do not think that cigarette prices and restaurant smoking are
# exogeneous in the income equation. income levels vary state-by-state,
# and its likely that legal restaurant smoking is associated with states
# that have lower income levels. also, cigarette prices are greatly
# impacted by taxes, and taxes are probably higher in states with low
# populations of smokers and high incomes.

results.summary(): 
                            OLS Regression Results                            
Dep. Variable:                lincome   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     39.61
Date:                Sat, 23 Apr 2022   Prob (F-statistic):           2.68e-30
Time:                        18:13:13   Log-Likelihood:                -798.50
No. Observations:                 807   AIC:                             1607.
Df Residuals:                     802   BIC:                             1630.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.7954      0.170

In [4]:
# ch17 q2
# the graduation probabily for someone who spends 10 hours per week in study hall is 99.9% 
# while someone who spend 5 hours has a 99.8% chance of graduating, making the estimated 
# different around 0.1%. 

In [68]:
# ch17 c9
a = woo.dataWoo('APPLE')
asummary = a.describe()
print(f'asummary:\n{asummary}\n')

# (i) 246 families report wanting none of the ecolabeled apples. 

# (ii) the variable ecolbs does seem to have continuous distribution over strictly 
# positive values, but also contains zero which means it would be suitable for a corner
# solution tobit model. 

y, X = pt.dmatrices('ecolbs ~ ecoprc + regprc + faminc + hhsize', data=a)
areg = smf.ols(formula='ecolbs ~ ecoprc + regprc + faminc + hhsize', data=a)
results_a = areg.fit()
sigma_start = np.log(sum(results_a.resid ** 2) / len(results_a.resid))
params_start = np.concatenate((np.array(results_a.params), sigma_start), axis=None)
class Tobit(smclass.GenericLikelihoodModel): 
    def nloglikeobs(self, params): 
        X = self.exog
        y = self.endog
        p = X.shape[1]
        beta = params[0:p]
        sigma = np.exp(params[p])
        y_hat = np.dot(X, beta)
        y_eq = (y == 0)
        y_g = (y > 0)
        l1 = np.empty(len(y))
        l1[y_eq] = np.log(stats.norm.cdf((-y_hat)[y_eq] / sigma))
        l1[y_g] = np.log(stats.norm.pdf((y - y_hat)[y_g] / sigma)) - np.log(sigma)
        return -l1
reg_tobit = Tobit(endog=y, exog=X)
results_tobit = reg_tobit.fit(start_params=params_start, maxiter=10000, disp=0)
print(f'results_tobit.summary(): \n{results_tobit.summary()}\n')

# (iii) the variables ecoprc and regprc are significant at the 1% significance level. 

hypotheses = ['faminc = hhsize']
ftest = results_a.f_test(hypotheses)
fstat = ftest.statistic[0][0]
fpval = ftest.pvalue
print(f'fstat: {fstat}\n')
print(f'fpval: {fpval}\n')

# (iv) the f-test p value is 0.428, so therefore faminc and hhsize are not jointly 
# significant. 

# (v) the signs of the coefficients on the prices variables from part iii are as expected
# since, assuming ecolabeled apples are a normal good, then price and quantity have an indirect 
# relationship. in response, the substitute good's, in this case regular apples would have a 
# direct relationship with the alternative's quantity because when the price of regular apples
# goes up, more people will then buy ecolabed ones instead. 

hypotheses = ['ecoprc = -regprc']
ftest = results_a.f_test(hypotheses)
fstat = ftest.statistic[0][0]
fpval = ftest.pvalue
print(f'fstat: {fstat}\n')
print(f'fpval: {fpval}\n')

# (vi) the p-value is 0.747, so we fail to reject the null hypothesis that B1 = -B2. 

new_vars = ['ecoprc_a', 'regprc_a', 'faminc_a', 'hhsize_a']
for i in range(len(new_vars)):
    a[new_vars[i]] = beta[i+1] * stats.norm.cdf(X @ beta / sigma)
print(f'a.head(): \n {a.head()}\n')
print(f'a.tail(): \n {a.tail()}\n')
print('means:')
print(np.mean(a[new_vars], axis=0))

# (vii) the smallest fitted value is 0.003640 and the largest is 3.101208. 

x_values = [-5.8215, 5.6552, 0.0066, 0.1303]
y_values = [-3.192412, 3.101208, 0.003640, 0.071465]
correlation_matrix = np.corrcoef(x_values, y_values)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2
print(r_squared)

# (viii) the squared correlation between ecolbs and ecolbshat is .99 repeated or 1. 

reg = smf.ols(formula='ecolbs ~ ecoprc + regprc + faminc + hhsize', data=a)
results = reg.fit()
print(f'results.summary(): \n{results.summary()}\n')

# (ix) the ols estimates are significantly smaller than the tobit estimates because
# the dataset contains a significant number of observations that equal zero, and the tobit
# model censors these values. in terms of goodness of fit, the tobit model is superior as its
# r-squared is 1 while the ols's is .039. 

# (x) had the r-squared from the tobit model been extremely small, the price still would
# not have been considered inconsistent because the tobit model's aim is to maximize
# log-liklihood, not the r-squared like in ols. 

asummary:
                 id        educ           date      regprc      ecoprc  \
count    660.000000  660.000000     660.000000  660.000000  660.000000   
mean   11729.009091   14.381818   68480.950000    0.882727    1.081515   
std     1071.582702    2.274014   50777.930716    0.244469    0.295573   
min    10002.000000    8.000000   10398.000000    0.590000    0.590000   
25%    10800.500000   12.000000   11998.000000    0.590000    0.890000   
50%    11692.000000   14.000000  111097.000000    0.890000    1.090000   
75%    12600.250000   16.000000  112397.000000    1.190000    1.290000   
max    13921.000000   20.000000  123197.000000    1.190000    1.590000   

         inseason      hhsize        male      faminc         age      reglbs  \
count  660.000000  660.000000  660.000000  660.000000  660.000000  660.000000   
mean     0.336364    2.940909    0.262121   53.409091   44.522727    1.282323   
std      0.472823    1.526049    0.440122   35.741220   15.212539    2.909862   

In [7]:
# ch17 c15
a = woo.dataWoo('ALCOHOL')
a.describe()

# (i) 89.8% of men in this sample were employed. 9.9% of men in this sample had 
# abused alcohol.

reg = smf.ols(formula='employ ~ abuse', data=a)
results = reg.fit(cov_type='HC3')
print(f'results.summary(): \n{results.summary()}\n')

# (ii) the regression equation implies that 90% of the men who do not abuse alcohol
# are employed, and that being a man who abuses alcohol makes you 2.83% less
# likely to be employed. the abuse coefficient is statistically significant,
# and I did expect it to have a negative sign.

reg_pro = smf.probit(formula='employ ~ abuse', data=a)
results_pro = reg_pro.fit(disp=0)
print(f'results_pro.summary(): \n{results_pro.summary()}\n')

coef_names = np.array(results.model.exog_names)
coef_names = np.delete(coef_names, 0)
APE_pro_autom = results_pro.get_margeff().margeff
table_auto = pd.DataFrame({'coef_names': coef_names,
                          'APE_pro_autom':
np.round(APE_pro_autom, 4)})
print(f'table_auto: \n{table_auto}\n')

# (iii) the sign and significance of the abuse coefficient are the same in the
# probit model. the partial effects in the probit model are very similar
# in the linear probability model.

X_new = pd.DataFrame(
    {'abuse': [0,1]})
predictions_lin = results.predict(X_new)
predictions_pro = results_pro.predict(X_new)
print(f'predictions_lin:\n {predictions_lin}\n')
print(f'predictions_pro:\n {predictions_pro}\n')

# (iv) the predictions are the same for the LPM and the probit model.

reg = smf.ols(formula='employ ~ abuse + age + agesq + educ + educsq + married \
                        + famsize + white + northeast + midwest + south + centcity + \
                        outercity + qrt1 + qrt2 + qrt3', data=a)
results = reg.fit(cov_type='HC3')
print(f'results.summary(): \n{results.summary()}\n')

# (v) he coefficient is slightly smaller, but it is statistically
# insignificant at the 5% level, so we assume that alcohol abuse
# does not impact employment likelihood.

reg1_pro = smf.probit(formula='employ ~ abuse  + age + agesq + educ + educsq + married + famsize + white + northeast + midwest + south + centcity + outercity + qrt1 + qrt2 + qrt3', data=alcohol)
results1_pro = reg1_pro.fit(disp=0)
print(f'results1_pro.summary(): \n{results1_pro.summary()}\n')

coef_names = np.array(results1.model.exog_names)
coef_names = np.delete(coef_names, 0)  # drop Intercept
APE_pro_autom = results1_pro.get_margeff().margeff
table_auto = pd.DataFrame({'coef_names': coef_names,
                          'APE_pro_autom':
np.round(APE_pro_autom, 4)})
print(f'table_auto: \n{table_auto}\n')

# (vi) the sign and significance of the abuse coefficient are the same in the
# probit model. the partial effects in the probit model are very similar
# in the linear probability model.
# the APE for the abuse coefficient is -0.0195, which is smaller than
# the LPM APE. however, the abuse coefficient in the probit model
# is statistically significant. the estimations are no longer similar,
# like they were in the simple regressions.

# (vii) health status could be related to disability and age, both of which affect
# employment ability. however, health status could also be correlated with
# alcohol abuse. a version of the model including health could be investigated
# to determine if health has any explanatory power.

# (viii) abuse could be an endogenous variable because people experiencing unemployment
# could use alcohol as a coping mechanism, leading to abuse. employment and
# alcohol abuse could be determined bidirectionally. mother and father alcohol
# abuse could be good instrumental variables because they could affect their
# child's likelihood of abusing alcohol, but it could also be correlated with
# the errors or with employment. testing is needed.

reg_iv = iv.IV2SLS.from_formula(formula='employ ~ 1 + [abuse ~ fathalc + mothalc] + age + agesq + educ + educsq + married + famsize + white + northeast + midwest + south + centcity + outercity + qrt1 + qrt2 + qrt3', data=alcohol)
results_iv = reg_iv.fit(cov_type='unadjusted', debiased=True)
table_iv = pd.DataFrame({'b': round(results_iv.params, 4),
                         'se': round(results_iv.std_errors, 4),
                         't': round(results_iv.tstats, 4),
                         'pval': round(results_iv.pvalues, 4)})
print(f'table_iv: \n{table_iv}\n')

#(ix) the instrumental variables coefficient is much larger than the LPM
# coefficient.

reg_redf = smf.ols(formula='abuse ~ fathalc + mothalc', data=a)
results_redf = reg_redf.fit()
a['resid'] = results_redf.resid
reg_secstg = smf.ols(formula='employ ~ abuse + resid', data=a)
results_secstg = reg_secstg.fit()

table_secstg = pd.DataFrame({'b': round(results_secstg.params, 4),
                             'se': round(results_secstg.bse, 4),
                             't': round(results_secstg.tvalues, 4),
                             'pval': round(results_secstg.pvalues, 4)})
print(f'table_secstg: \n{table_secstg}\n')

# (x) the residuals have a p-value under .10, so they are just barely
# significant at the 10% and indicates that there could be
# endogeneity.

results.summary(): 
                            OLS Regression Results                            
Dep. Variable:                 employ   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.441
Date:                Sat, 23 Apr 2022   Prob (F-statistic):             0.0112
Time:                        18:10:03   Log-Likelihood:                -2185.9
No. Observations:                9822   AIC:                             4376.
Df Residuals:                    9820   BIC:                             4390.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9010      0.003